## SIMD JULIA

In [19]:
struct VecElement{T}
    value::T
end

In [76]:
const m128 = NTuple{4,VecElement{Float32}}

function add(a::m128, b::m128)
    (VecElement(a[1].value+b[1].value),
     VecElement(a[2].value+b[2].value),
     VecElement(a[3].value+b[3].value),
     VecElement(a[4].value+b[4].value))
end



add (generic function with 2 methods)

In [72]:
code_native(add, (m128, m128))

	.section	__TEXT,__text,regular,pure_instructions
	.build_version macos, 12, 0
	.globl	_julia_add_3192                 ; -- Begin function julia_add_3192
	.p2align	2
_julia_add_3192:                        ; @julia_add_3192
; ┌ @ In[71]:3 within `add`
	.cfi_startproc
; %bb.0:                                ; %top
; │ @ In[71]:4 within `add`
; │┌ @ float.jl:383 within `+`
	ldr	q0, [x0]
	ldr	q1, [x1]
	fadd	v0.4s, v0.4s, v1.4s
; │└
	str	q0, [x8]
	ret
	.cfi_endproc
; └
                                        ; -- End function
.subsections_via_symbols


In [48]:
v = m128(VecElement.(Float32.((2,2,4,5))))

(VecElement{Float32}(2.0f0), VecElement{Float32}(2.0f0), VecElement{Float32}(4.0f0), VecElement{Float32}(5.0f0))

In [29]:
x = Float32.((2,2,4,5))

(2.0f0, 2.0f0, 4.0f0, 5.0f0)

In [30]:
using BenchmarkTools

In [32]:
@benchmark x.+x

BenchmarkTools.Trial: 10000 samples with 455 evaluations.
 Range (min … max):  231.136 ns …   3.518 μs  ┊ GC (min … max): 0.00% … 90.43%
 Time  (median):     238.187 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   243.758 ns ± 109.795 ns  ┊ GC (mean ± σ):  1.52% ±  3.14%

     ▁█▄▂                                                        
  ▁▂▃████▇▅▃▂▃▃▃▅▅█▇▅▄▄▃▂▂▃▂▂▂▃▃▃▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  231 ns           Histogram: frequency by time          267 ns <

 Memory estimate: 128 bytes, allocs estimate: 3.

In [36]:
@benchmark add($v,$v)

BenchmarkTools.Trial: 10000 samples with 1000 evaluations.
 Range (min … max):  2.084 ns … 23.459 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.167 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.212 ns ±  0.401 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

      ▁    █    ▇    ▄    ▂   ▂    ▁    ▁                    ▁
  ▃▁▁▁█▁▁▁▁█▁▁▁▁█▁▁▁▁█▁▁▁▁█▁▁▁█▁▁▁▁█▁▁▁▁█▁▁▁▁▇▁▁▁▁▇▁▁▁▁▆▁▁▁▇ █
  2.08 ns      Histogram: log(frequency) by time     2.58 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [37]:
using SIMD

In [41]:
v_vec = Vec( Float32.((2,2,4,5)))

<4 x Float32>[2.0, 2.0, 4.0, 5.0]

In [47]:
@benchmark $v_vec + $v_vec

BenchmarkTools.Trial: 10000 samples with 1000 evaluations.
 Range (min … max):  2.125 ns … 7.375 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.167 ns             ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.187 ns ± 0.084 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

             █                                               
  ▂▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▇▄▁▁▁▁▁▁▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▂ ▂
  2.12 ns        Histogram: frequency by time       2.33 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [44]:
@code_llvm add(v,v)

;  @ In[20]:3 within `add`
define void @julia_add_2810([4 x [1 x float]]* noalias nocapture sret([4 x [1 x float]]) %0, [4 x [1 x float]]* nocapture nonnull readonly align 4 dereferenceable(16) %1, [4 x [1 x float]]* nocapture nonnull readonly align 4 dereferenceable(16) %2) #0 {
top:
;  @ In[20]:4 within `add`
; ┌ @ float.jl:383 within `+`
   %3 = bitcast [4 x [1 x float]]* %1 to <4 x float>*
   %4 = load <4 x float>, <4 x float>* %3, align 4
   %5 = bitcast [4 x [1 x float]]* %2 to <4 x float>*
   %6 = load <4 x float>, <4 x float>* %5, align 4
   %7 = fadd <4 x float> %4, %6
; └
  %8 = bitcast [4 x [1 x float]]* %0 to <4 x float>*
  store <4 x float> %7, <4 x float>* %8, align 4
  ret void
}


In [45]:
@code_llvm v_vec + v_vec

;  @ /Users/dbuchaca/.julia/packages/SIMD/myoU9/src/simdvec.jl:252 within `+`
define void @"julia_+_2812"([1 x <4 x float>]* noalias nocapture sret([1 x <4 x float>]) %0, [1 x <4 x float>]* nocapture nonnull readonly align 16 dereferenceable(16) %1, [1 x <4 x float>]* nocapture nonnull readonly align 16 dereferenceable(16) %2) #0 {
top:
;  @ /Users/dbuchaca/.julia/packages/SIMD/myoU9/src/simdvec.jl:253 within `+`
; ┌ @ Base.jl:38 within `getproperty`
   %3 = getelementptr inbounds [1 x <4 x float>], [1 x <4 x float>]* %1, i64 0, i64 0
   %4 = getelementptr inbounds [1 x <4 x float>], [1 x <4 x float>]* %2, i64 0, i64 0
; └
; ┌ @ /Users/dbuchaca/.julia/packages/SIMD/myoU9/src/LLVM_intrinsics.jl:212 within `fadd` @ /Users/dbuchaca/.julia/packages/SIMD/myoU9/src/LLVM_intrinsics.jl:212
; │┌ @ /Users/dbuchaca/.julia/packages/SIMD/myoU9/src/LLVM_intrinsics.jl:221 within `macro expansion`
    %5 = load <4 x float>, <4 x float>* %3, align 16
    %6 = load <4 x float>, <4 x float>* %4, align 16

Note that we can leverage 128 bit registers with other datatypes, such as int8

In [111]:
const i8_128 = NTuple{16,VecElement{Int8}}

function add(a::i8_128, b::i8_128)
    (VecElement(a[1].value + b[1].value),
     VecElement(a[2].value + b[2].value),
     VecElement(a[3].value + b[3].value),
     VecElement(a[4].value + b[4].value),
     VecElement(a[5].value + b[5].value),
     VecElement(a[6].value + b[6].value),
     VecElement(a[7].value + b[7].value),
     VecElement(a[8].value + b[8].value),
     VecElement(a[9].value + b[9].value),
     VecElement(a[10].value + b[10].value),
     VecElement(a[11].value + b[11].value),
     VecElement(a[12].value + b[12].value),
     VecElement(a[13].value + b[13].value),
     VecElement(a[14].value + b[14].value),
     VecElement(a[15].value + b[15].value),
     VecElement(a[16].value + b[16].value))
end

v = VecElement.(Int8.((1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16)))

@benchmark add($v,$v)

BenchmarkTools.Trial: 10000 samples with 1000 evaluations.
 Range (min … max):  2.083 ns … 18.792 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.167 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.199 ns ±  0.346 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

          ▁       █       ▆▅       ▃       ▂       ▂         ▁
  ▃▁▁▁▁▁▁▁█▁▁▁▁▁▁▁█▁▁▁▁▁▁▁██▁▁▁▁▁▁▁█▁▁▁▁▁▁▁█▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▅ █
  2.08 ns      Histogram: log(frequency) by time     2.38 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

Note that we might encouter with overflow and underflow issues:

Here we can see that 126 + 3 = 129 bet we get -127

In [112]:
vmax = VecElement.(Int8.((124,125,126,127,5,6,7,8,9,10,11,12,13,14,15,-128)))
v = VecElement.(Int8.((1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1)))

add(v,vmax)

(VecElement{Int8}(125), VecElement{Int8}(126), VecElement{Int8}(127), VecElement{Int8}(-128), VecElement{Int8}(6), VecElement{Int8}(7), VecElement{Int8}(8), VecElement{Int8}(9), VecElement{Int8}(10), VecElement{Int8}(11), VecElement{Int8}(12), VecElement{Int8}(13), VecElement{Int8}(14), VecElement{Int8}(15), VecElement{Int8}(16), VecElement{Int8}(-127))